**Zadanie 1**  
Wczytaj plik `zamowienia.csv` do ramki pandas, a następnie w kilku miejscach (ale nie w pierwszych 10 wierszach) wstaw wartość NaN, aby zasymulować wartości brakujące. Zapisz ramkę do pliku `zamowienia_missing.csv`. Wczytaj teraz plik do ramki Dask i sprawdź jakie typy danych zostały przydzielone. Czy zgadzają się z typami z oryginalnego pliku? Wykonaj dowolne obliczenia na całej ramce Dask, aby wymusić wywołanie `.compute()`. Czy pojawił się błąd dotyczący niespójności typów danych? Spróbuj uruchomić kilka razy funkcję wczytywania danych do ramki Dask dataframe z różnymi wartościami parametru `samples`. Dokumentacja `dask.dataframe.read_csv()`: [https://docs.dask.org/en/stable/generated/dask.dataframe.read_csv.html](https://docs.dask.org/en/stable/generated/dask.dataframe.read_csv.html)

In [1]:
import pandas as pd
import numpy as np

zamowienia = pd.read_csv('zamowienia.csv')
# zamowienia.to_csv('zamowienia_missing.csv', index=False)

In [8]:
import dask.dataframe as dd

ddf = dd.read_csv('zamowienia_missing.csv', sep=';')
print(ddf.dtypes)
ddf.compute()

Kraj               string[pyarrow]
Sprzedawca         string[pyarrow]
Data zamowienia    string[pyarrow]
idZamowienia                 int64
Utarg                      float64
dtype: object


ValueError: Mismatched dtypes found in `pd.read_csv`/`pd.read_table`.

+--------------+---------+----------+
| Column       | Found   | Expected |
+--------------+---------+----------+
| idZamowienia | float64 | int64    |
+--------------+---------+----------+

Usually this is due to dask's dtype inference failing, and
*may* be fixed by specifying dtypes manually by adding:

dtype={'idZamowienia': 'float64'}

to the call to `read_csv`/`read_table`.

Alternatively, provide `assume_missing=True` to interpret
all unspecified integer columns as floats.

In [10]:
ddf[ddf['Kraj'] == 'Polska']['Utarg'].sum().compute()

np.float64(332496.70999999996)

Samo `.compute()` podaje błąd ze względu na brakujące wartości, w kolumnie `'Utarg'` również są wartości NaN.

In [ ]:
# samples

**Zadanie 2**  
Ze strony [https://docs.dask.org/en/stable/dashboard.html](https://docs.dask.org/en/stable/dashboard.html) skonfiguruj plugin Dask dashboard dla Jupyter Lab i przetestuj jego działanie.

In [11]:
from dask.distributed import Client
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 4,Total memory: 15.84 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:53112,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 15.84 GiB
Comm: tcp://127.0.0.1:53134,Total threads: 1
Dashboard: http://127.0.0.1:53135/status,Memory: 3.96 GiB
Nanny: tcp://127.0.0.1:53115,


In [ ]:
# przetestuj działanie

**Zadanie 3**  
Skonfiguruj lokalny klaster (`Client`) tak, aby nie zaalokował wszystkich zasobów (np. zostaw 8 GB RAM dla systemu hosta + 2 rdzenie). Pobierz dane udostępnione na poprzednich zajęciach ([https://huggingface.co/datasets/vargr/private_instagram/tree/main/data](https://huggingface.co/datasets/vargr/private_instagram/tree/main/data)) i załaduj do ramki Dask tyle części ile zdołasz w formie bez optymalizacji. Zmierz czas tej operacji.

In [ ]:
client = Client(n_workers=4, threads_per_worker=2, memory_limit='8GB')

ddf_instagram = dd.read_csv(f'{data_path}/*.csv')

# Zmierz czas ładowania danych
import time
start_time = time.time()

# Wczytaj dane bez optymalizacji
ddf_instagram.head()  # Wczytaj próbkę

end_time = time.time()
print(f"Czas ładowania danych: {end_time - start_time} sekund")


**Zadanie 4**  
Wykonaj kilka operacji na klastrze lokalnym z danymi z zadania 3:

- wyświetl top 10 użytkowników z najwyższą liczbą like'ów,
- pobierz dane tylko za pierwsze półrocze 2019 roku. Każdorazowo zmierz i wyświetl czas operacji i obserwuj dashboard.

In [ ]:
# dodać dane z lab01 - tylko na stacjonarnym - tutaj dać wszystko co się da z instagram

**Zadanie 5**  
Wczytaj te same dane do ramki Dask co w zadaniu 3, ale podaj typy danych, które zostały wybrane w procesie optymalizacji wykonanej w zadaniach z lab 01. Porównaj czas ładowania z zadaniem 3. Wykonaj również te same operacje co w zadaniu 4 i porównaj czas. Śledź wykonanie zadań patrząc na graf wywołań.

**Zadanie 6**  
Podziel tablicę `darr` z przykładów na inne liczby chunków (eksperymentuj) i wykonaj te same obliczenie (średnia). Dla każdej liczby chunków wypisz czas obliczeń (wykonaj to samo obliczenie minimum 10 razy, aby nieco uwiarygodnić wyniki i uśrednij) i porównaj wyniki. Napisz wniosek o wynikach swoich eksperymentów i automatycznego podziału na chunki. Czy udało Ci się osiągnąć lepszą wydajność niż przy domyślnych ustawieniach?